In [2]:
"""Gets caltrain timetable and processes it into a JSON"""

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [3]:
stuff = (
    ("weekday", "https://www.caltrain.com/schedules/weekdaytimetable.html", "NB_TT", "Northbound Train No."),
    ("weekday", "https://www.caltrain.com/schedules/weekdaytimetable.html", "SB_TT", "Southbound Train No."),
    # ("weekend", "https://www.caltrain.com/schedules/weekend-timetable.html", "NB_TT", "Train No."),
    # ("weekend", "https://www.caltrain.com/schedules/weekend-timetable.html", "SB_TT", "Train No."),
)


def get_table(schedule_type, url, table_class, col_name):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    html = soup.find("table", {"class": table_class})
    df = pd.read_html(str(html))[0].transpose()

    # clean up nans and weird characters
    df = df.replace(np.NaN, "-")
    df = df.replace("–", "-")

    # do index stuff
    if schedule_type == "weekday":
        df = df[1:]
        df = df.reset_index(level=0)
        df = df.rename(columns=df.iloc[0])
    elif schedule_type == "weekend":
        df = df[1:]

    # drop unneeded columns + rows
    df = df[[x[0] not in "NSZ" for x in df[col_name]]]
    df = df.drop("Service Type", 1)

    # get a dict
    js = df.to_dict(orient="split")
    del js["index"]
    return js
    
# for schedule_type, url, table_class, col_name in stuff:
#     print("\n\n\n\n")
#     print(get_table(url, table_class, col_name))


get_table(*stuff[0])

# for schedule_type, url, table_class, col_name in stuff:
#     print("\n\n\n\n")


ValueError: No tables found